In [4]:
# Initial imports
import warnings
warnings.filterwarnings('ignore')
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
# Initialize the Panel Extensions (for Plotly)
import panel as pn
import param
pn.extension('plotly')
pn.extension()
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline
#!pip install yfinance
import yfinance as yf
#!pip install pandas-montecarlo
import pandas_montecarlo
import datetime as dt
#from MCForecastTools import MCSimulation
# Load .env enviroment variables
load_dotenv()



True

In [5]:
# Define Parameters
# Set number of simulations for MC
num_sims = 500
range_of_cryptos = ["BTC-USD", "ETH-USD", "BNB-USD", "ADA-USD", "DOGE-USD", "XRP-USD", "LINK-USD" , "THETA-USD", "LTC-USD", "XLM-USD"]
range_of_stocks = ["AMZN", "TSLA", "NEO", "AAPL", "NVDA", "BABA", "NFLX", "DIS", "NKE"]

# Define Functions

#CALCULATE 1 YEARS BACK DATE
def calc_1years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=365)
    return start
#CALCULATE 2 YEARS BACK DATE
def calc_2years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=730)
    return start
#CALCULATE 3 YEARS BACK DATE
def calc_3years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=1095)
    return start


#download crypto data and stock data for 3 years
def download_cryptos_data():
    df = yf.download(range_of_cryptos, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df

def download_stocks_data():
    df = yf.download(range_of_stocks, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df




def cumulative_return_plot(df):
    cumulative_returns = (1 + df['Close'].pct_change()).cumprod()
    plot = cumulative_returns.hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Time", ylabel="Price", title="Cumulative Return")
    return plot


def MA_plot(df):
    df["30MA"]=df[["Close"]].rolling(30).mean()
    df["60MA"]=df[["Close"]].rolling(60).mean()
    df["90MA"]=df[["Close"]].rolling(90).mean()
    df.dropna(inplace=True)
    plot = df[["Close", "30MA", "60MA", "90MA"]].hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Date", ylabel="Price", title="Moving Average")   
    return plot

def daily_returns_multiple_tickers(df):
    df2 = pd.DataFrame(columns = df.columns.levels[0])
    df = df.pct_change()
    df = df.dropna()
    for key in (df.columns.levels[0]):
        df2[key] =+ (df[key]["Close"])
    return df2  

def annual_sharpe_ratio_plot(df):
    df = daily_returns_multiple_tickers(df)
    std = df.std()
    annual_std = std * np.sqrt(252)
    annual_sharpe_ratio = (df.mean() * 252) / annual_std
    plot = annual_sharpe_ratio.hvplot(kind="bar", rot=45, shared_axes=False).opts(width=500, xlabel="Date", ylabel="Price", title="Annual Sharpe Ratios")
    return plot

##Monte Carlo plot:
##annual return function:
def daily_return_data(df):
    daily_return = df["Close"].pct_change()
    return daily_return
##Monte Carlo function:
def MC_100simulation_plot(df):
    mc = MCSimulation(
        portfolio_data = df,
        weights = "",
        num_simulation = 100,
        num_trading_days = 252*5
)

    return plot




In [53]:
mc = MCSimulation(
    portfolio_data = crypto_df,
    weights = "",
    num_simulation = 100,
    num_trading_days = 252*5
)
mc.portfolio_data.head()


BTC-USD                                                      \
                   Open         High          Low        Close    Adj Close   
Date                                                                          
2018-07-08  6857.799805  6885.910156  6747.979980  6773.879883  6773.879883   
2018-07-09  6775.080078  6838.680176  6724.339844  6741.750000  6741.750000   
2018-07-10  6739.209961  6767.740234  6320.720215  6329.950195  6329.950195   
2018-07-11  6330.770020  6444.959961  6330.470215  6394.709961  6394.709961   
2018-07-12  6396.779785  6397.100098  6136.419922  6228.810059  6228.810059   

                                     DOGE-USD                      ...  \
                Volume daily_return      Open      High       Low  ...   
Date                                                               ...   
2018-07-08  3386210048          NaN  0.002642  0.002671  0.002573  ...   
2018-07-09  3718129920    -0.004743  0.002612  0.002649  0.002595  ...   
2018-07-10  4052430080    -0.061082  0.002606  0.002623  0.002242  ...   
2018-07-11  3644859904     0.010231  0.002406  0.002475  0.002353  ...   
2018-07-12  3770170112    -0.025943  0.002395  0.002425  0.002244  ...   

             BNB-USD                          ADA-USD                      \
           Adj Close    Volume daily_return      Open      High       Low   
Date                                                                        
2018-07-08   13.9919  60275400          NaN  0.149953  0.151385  0.144762   
2018-07-09   13.6645  36874000    -0.023399  0.145345  0.146111  0.139948   
2018-07-10   12.7613  36507100    -0.066098  0.139729  0.140654  0.128792   
2018-07-11   12.7049  38867700    -0.004420  0.128801  0.133925  0.126466   
2018-07-12   12.0706  26008100    -0.049926  0.131560  0.131560  0.123154   

                                                       
               Close Adj Close    Volume daily_return  
Date                                                   
2018-07-08  0.145389  0.145389  48142400          NaN  
2018-07-09  0.139971  0.139971  42768500    -0.037266  
2018-07-10  0.128792  0.128792  62318800    -0.079867  
2018-07-11  0.131483  0.131483  50179700     0.020894  
2018-07-12  0.127181  0.127181  37623900    -0.032719  

[5 rows x 70 columns]

In [6]:
# Create a Title for the Dashboard

# Fetch initial data:
crypto_df = download_cryptos_data()
stock_df = download_stocks_data()
fetch_data_button = pn.widgets.Button(name='FETCH HISTORICAL DATA FROM YAHOO FINANCE!', button_type='success')
def update_data(event):
    crypto_df = download_cryptos_data()
    stock_df = download_stocks_data()
fetch_data_button.on_click(update_data)



# Define title and welcome text
title = "#Investment Projection Dasboard"
welcome = "This Dashboard provice a visual analysis of historial value for certain stock and cryptos"


#Create Widget Elements:
crypto_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_year_slider = pn.widgets.IntSlider(name='Years of Crypto Data', start=1, end=3, step=1, value=1)
stock_ticker = pn.widgets.Select(name='Stock Tickers', options=range_of_stocks) 
stock_year_slider = pn.widgets.IntSlider(name='Years of Stock Data', start=1, end=3, step=1, value=1)

# Input for Historical data
#Crypto historical Data:
crypto_input = pn.Column(crypto_ticker, crypto_year_slider, align='center')
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_MAchart(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(crypto_df[crypto_ticker])
    
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_cumulative(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(crypto_df[crypto_ticker])
crypto_column = pn.Column(crypto_input, reactive_crypto_MAchart, reactive_crypto_cumulative)


#Stock historical Data:
stock_input = pn.Column(stock_ticker, stock_year_slider, align='center')
@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_MAchart(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return MA_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return MA_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(stock_df[stock_ticker])

@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_cumulative(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(stock_df[stock_ticker])
stock_column = pn.Column(stock_input, reactive_stock_MAchart, reactive_stock_cumulative)

historical_data_row = pn.Row(crypto_column, stock_column, width=1100)


#Input for your proforlio
crypto_investment_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_investment = pn.widgets.FloatInput(name='Your Crypto Investment in USD', value=0)
crypto_future_slider = pn.widgets.IntSlider(name='Years of MC projection', start=2, end=5, step=3, value=2)

stock_investment_ticker = pn.widgets.Select(name='Stock Tickes', options=range_of_cryptos)
stock_investment = pn.widgets.FloatInput(name='Your Stock Investment in USD', value=0)
stock_future_slider = pn.widgets.IntSlider(name='Years of MC projection', start=2, end=5, step=3, value=2)

cal_crypto_MC_button = pn.widgets.Button(name='Calculate MC', button_type='danger')
#@pn.depends(crypto_investment_ticker)
#def calc_crypto_MC_data(crypto_investment_ticker):
    #
    #
    #
    #
#cal_crypto_MC_button.on_click(calc_crypto_MC_data)
cal_stock_MC_button = pn.widgets.Button(name='Calculate MC', button_type='danger')
#@pn.depends(stock_investment, stock_future_slider)
#def calc_stock_MC_data(event):
    #
    #CODING LINE HERE!
    #
    #
#cal_stock_MC_button.on_click(calc_stock_MC_data)

crypto_user_input = pn.Column(crypto_investment_ticker, crypto_investment, crypto_future_slider, cal_crypto_MC_button, align='center')
stock_user_input = pn.Column(stock_investment_ticker, stock_investment, stock_future_slider, cal_stock_MC_button, align='center')
user_input = pn.Row(crypto_user_input, stock_user_input)






In [7]:
# Create a layout for the dashboard
dashboard = pn.WidgetBox(pn.Column(title, fetch_data_button,
                        pn.WidgetBox(pn.Tabs(    
                            ("Historical Data", historical_data_row),
                            ("MC Projection Alnalysis", user_input),
                            )), align="center"))

dashboard.servable()


WidgetBox
    [0] Column(align='center')
        [0] Markdown(str)
        [1] Button(button_type='success', name='FETCH HISTORICAL D...)
        [2] WidgetBox
            [0] Tabs
                [0] Row(width=1100)
                    [0] Column
                        [0] Column(align='center')
                            [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                            [1] IntSlider(end=3, name='Years of Crypto Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                    [1] Column
                        [0] Column(align='center')
                            [0] Select(name='Stock Tickers', options=['AMZN', 'TSLA', ...], value='AMZN')
                            [1] IntSlider(end=3, name='Years of Stock Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                [1] Row
                    [0] Column(align='center')
                        [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                        [1] FloatInput(name='Your Crypto I...)
                        [2] IntSlider(end=5, name='Years of MC projection', start=2, step=3, value=2)
                        [3] Button(button_type='danger', name='Calculate MC')
                    [1] Column(align='center')
                        [0] Select(name='Stock Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                        [1] FloatInput(name='Your Stock Investment i...)
                        [2] IntSlider(end=5, name='Years of MC projection', start=2, step=3, value=2)
                        [3] Button(button_type='danger', name='Calculate MC')

In [9]:
"""MC_2years = MCSimulation(
    portfolio_data = crypto_df,
    weights = "",
    num_simulation = 500,
    num_trading_days = 252*2
)
MC_2years.plot_simulation()"""

'MC_2years = MCSimulation(\n    portfolio_data = crypto_df,\n    weights = "",\n    num_simulation = 500,\n    num_trading_days = 252*2\n)\nMC_2years.plot_simulation()'

In [141]:
def daily_returns_multiple_tickers(df):
    df2 = pd.DataFrame(columns = df.columns.levels[0])
    df = df.pct_change()
    df = df.dropna()
    for key in (df.columns.levels[0]):
        df2[key] =+ (df[key]["Close"])
    return df2  

def annual_sharpe_ratio_plot(df):
    df = daily_returns_multiple_tickers(df)
    std = df.std()
    annual_std = std * np.sqrt(252)
    annual_sharpe_ratio = (df.mean() * 252) / annual_std
    plot = annual_sharpe_ratio.hvplot(kind="bar", rot=45, shared_axes=False).opts(width=500, xlabel="Date", ylabel="Price", title="Annual Sharpe Ratios")
    return plot

annual_sharpe_ratio_plot(stock_df)

:Bars   [index]   (0)

THETA-USD  LINK-USD   LTC-USD   BNB-USD  DOGE-USD   XLM-USD  \
Date                                                                      
2018-07-09  -0.087050 -0.046724 -0.018671 -0.023399 -0.003064 -0.008838   
2018-07-10  -0.096822 -0.102242 -0.059043 -0.066098 -0.073761 -0.077142   
2018-07-11  -0.005168 -0.020988  0.034388 -0.004420  0.005807 -0.012429   
2018-07-12  -0.005645 -0.010356 -0.024032 -0.049926 -0.049072 -0.019973   
2018-07-13  -0.022435 -0.019057  0.001743  0.009072  0.011275  0.099898   
...               ...       ...       ...       ...       ...       ...   
2021-07-04   0.012465  0.039054  0.032978  0.031837  0.000292  0.014391   
2021-07-05  -0.063729 -0.046268 -0.047152 -0.017399 -0.060325 -0.050991   
2021-07-06   0.030982  0.093017  0.006608  0.061370  0.012124  0.022534   
2021-07-07   0.039131 -0.015753 -0.011616  0.020526 -0.043921 -0.012491   
2021-07-08   0.009945 -0.050975 -0.049271 -0.036302 -0.048479 -0.052591   

             ETH-USD   ADA-USD   XRP-USD   BTC-USD  
Date                                                
2018-07-09 -0.025427 -0.037266 -0.007282 -0.004743  
2018-07-10 -0.088650 -0.079867 -0.061627 -0.061082  
2018-07-11  0.027839  0.020894  0.011664  0.010231  
2018-07-12 -0.036827 -0.032719 -0.033025 -0.025943  
2018-07-13  0.009191  0.084179  0.007757  0.001483  
...              ...       ...       ...       ...  
2021-07-04  0.042949  0.036499  0.032780  0.017862  
2021-07-05 -0.053039 -0.036543 -0.058487 -0.043692  
2021-07-06  0.057354  0.009364  0.016968  0.014496  
2021-07-07 -0.004094 -0.010532 -0.020780 -0.011096  
2021-07-08 -0.069838 -0.026084 -0.046475 -0.038672  

[1089 rows x 10 columns]